# 1. Set Up DF

In [1]:
import pandas as pd
import numpy as np
import re
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

## 1.1 Create look-up table "Team"

In [2]:
teamName = ['Danang-Dragons','Hanoi-Buffalos','Can-Tho-Catfish','Hochiminh-City-Wings','Saigon-Heat','Thang-Long-Warriors']
teamID = ['DND','HNB','CTC','HCW','SGH','TLW']
teamDict = {teamName[i]:teamID[i] for i in range(len(teamName))}
teamDict2 = {teamID[i]:teamName[i] for i in range(len(teamName))}
df_team = pd.DataFrame({"teamID":teamID,"teamName":teamName})
df_team

,teamID,teamName
0,DND,Danang-Dragons
1,HNB,Hanoi-Buffalos
2,CTC,Can-Tho-Catfish
3,HCW,Hochiminh-City-Wings
4,SGH,Saigon-Heat
5,TLW,Thang-Long-Warriors


## 1.2 Create look-up table "Round"

In [3]:
roundID = []
roundName = []
roundType = []
roundYear = []
roundNum = []

roundDict = {2017:{},2018:{},2019:{}}

for year in [2017,2018,2019]:
    for num in range(1,16):
        roundID.append(str(year) + 'REG' + str(num).zfill(2))
        roundName.append('Regular '+str(num))
        roundType.append('Regular')
        roundYear.append(year)
        roundNum.append(num)
        roundDict[year]['Regular '+str(num)]=(str(year) + 'REG' + str(num).zfill(2))
        if (num<=5):
            roundID.append(str(year) + 'SEM' + str(num).zfill(2))
            roundName.append('Semi-Finals '+str(num))
            roundType.append('Semi-Finals')
            roundYear.append(year)
            roundNum.append(num)
            roundDict[year]['Semi-Finals '+str(num)]=(str(year) + 'SEM' + str(num).zfill(2))
            
            roundID.append(str(year) + 'FIN' + str(num).zfill(2))
            roundName.append('Finals '+str(num))
            roundType.append('Finals')
            roundYear.append(year)
            roundNum.append(num)
            roundDict[year]['Finals '+str(num)]=(str(year) + 'FIN' + str(num).zfill(2))

df_round = pd.DataFrame({"roundID":roundID,
                         "roundName":roundName,
                         "roundYear":roundYear,
                         "roundType":roundType,
                         "roundNum":roundNum}).sort_values(by="roundID").reset_index(drop=True)

df_round.head()

,roundID,roundName,roundYear,roundType,roundNum
0,2017FIN01,Finals 1,2017,Finals,1
1,2017FIN02,Finals 2,2017,Finals,2
2,2017FIN03,Finals 3,2017,Finals,3
3,2017FIN04,Finals 4,2017,Finals,4
4,2017FIN05,Finals 5,2017,Finals,5


## 1.3 Concatenate & Normalise 3 years data for team

In [4]:
result_team = pd.DataFrame(columns=["roundID","teamID","oppID","fgm","fga","x3pm","x3pa","ftm","fta","oreb","dreb","ast","stl","blk","tov","pf","pts","win"])
#Help function
for year in [2017,2018,2019]:
    x = pd.read_csv("vba"+str(year)+"_team_data_final.csv").drop('Unnamed: 0',axis=1)
    #Add WIN column
    x['WIN']=0
    with pd.option_context('mode.chained_assignment', None):
        for i in range(0,x.shape[0],2):
            if (int(x['PTS'][i:i+1]) > int(x['PTS'][i+1:i+2])):
                x['WIN'][i:i+1] = 1
            else:
                x['WIN'][i+1:i+2] = 1
        
    x['TEAM'] = x['TEAM'].map(teamDict)
    x['OPP'] = x['OPP'].map(teamDict)
    x['ROUND'] = x['ROUND'].map(roundDict[year])

    x = x.rename(str.lower, axis=1)
    x = x.rename({"team":"teamID","round":"roundID","opp":"oppID","3pm":"x3pm","3pa":"x3pa","to":"tov"},axis=1)
    result_team = pd.concat([result_team,x]).reset_index(drop=True)

result_team.head(6)

,roundID,teamID,oppID,fgm,fga,x3pm,x3pa,ftm,fta,oreb,dreb,ast,stl,blk,tov,pf,pts,win
0,2017REG01,DND,TLW,20,74,8,26,17,31,10,19,8,5,2,26,13,65,0
1,2017REG01,TLW,DND,40,85,3,17,13,17,5,22,23,8,1,21,26,96,1
2,2017REG01,CTC,HNB,34,70,10,20,5,14,7,25,32,11,5,21,17,83,0
3,2017REG01,HNB,CTC,31,71,12,29,12,16,11,30,27,7,2,24,14,86,1
4,2017REG01,SGH,TLW,30,64,10,22,16,30,6,31,21,14,3,17,22,86,1
5,2017REG01,TLW,SGH,29,73,7,21,13,25,13,31,20,8,1,28,21,78,0


## 1.4 Concatenate & Normalise 3 years data for players

In [5]:
result_player = pd.DataFrame(columns=["roundID","teamID","oppID","#","name","min","fgm","fga","x3pm","x3pa","ftm","fta","oreb","dreb","ast","stl","blk","tov","pf","pts"])
#Help function
for year in [2017,2018,2019]:
    x = pd.read_csv("vba"+str(year)+"_player_data_final.csv").drop('Unnamed: 0',axis=1)
        
    x['TEAM'] = x['TEAM'].map(teamDict)
    x['OPP'] = x['OPP'].map(teamDict)
    x['ROUND'] = x['ROUND'].map(roundDict[year])

    x = x.rename(str.lower, axis=1)
    x = x.rename({"team":"teamID","round":"roundID","opp":"oppID","3pm":"x3pm","3pa":"x3pa","to":"tov"},axis=1)
    result_player = pd.concat([result_player,x]).reset_index(drop=True)

result_player.head()

,roundID,teamID,oppID,#,name,min,fgm,fga,x3pm,x3pa,ftm,fta,oreb,dreb,ast,stl,blk,tov,pf,pts
0,2017REG01,DND,TLW,12,"Nguyen, Tuan",35,6,15,3,9,1,2,0,1,2,0,0,7,0,16
1,2017REG01,DND,TLW,0,"Nguyen, Horace",27,4,18,4,10,3,6,0,0,0,2,0,3,1,15
2,2017REG01,DND,TLW,25,"Joly, Rudolphe",27,4,10,0,0,3,6,6,10,2,1,0,3,1,11
3,2017REG01,DND,TLW,34,"Tim, Waale",21,1,4,0,0,2,6,2,2,0,1,1,2,3,4
4,2017REG01,DND,TLW,11,"Thinh Tran, Tien",18,4,7,1,1,5,5,2,1,0,0,1,2,2,14


# 2. Regular season stats

## 2.1 For Team

### 2.1.1 Basic Stats
Total Game Played, Total Win, Average Stats

In [6]:
x = pd.merge(result_team,df_round,on="roundID")
stats = ["fgm","fga","x3pm","x3pa","ftm","fta","oreb","dreb","ast","stl","blk","tov","pf","pts"]

gk = x[x["roundType"]=="Regular"].groupby(["teamID","roundYear"])
df = pd.DataFrame(columns=["teamID","year","gp","win"]+stats)
for k, group in gk:
    y = group[stats].mean()
    y["gp"]=group.shape[0]
    y["win"]=group["win"].sum()
    y["teamID"]=k[0]
    y["year"]=k[1]
    df = df.append(y,ignore_index=True)

df

,teamID,year,gp,win,fgm,fga,x3pm,x3pa,ftm,fta,oreb,dreb,ast,stl,blk,tov,pf,pts
0,CTC,2017,14.0,8.0,31.928571,75.071429,8.357143,27.285714,8.785714,16.214286,12.000000,28.571429,21.785714,11.571429,4.428571,23.142857,15.785714,81.000000
1,CTC,2018,15.0,13.0,32.733333,73.866667,10.800000,33.133333,9.466667,16.000000,11.333333,26.933333,18.066667,11.600000,3.600000,15.266667,15.533333,85.733333
2,CTC,2019,14.0,9.0,33.500000,76.214286,7.714286,28.000000,10.857143,18.857143,10.857143,32.642857,15.571429,12.214286,3.642857,17.785714,15.214286,85.571429
3,DND,2017,13.0,2.0,26.692308,67.615385,5.230769,20.230769,11.846154,18.923077,9.923077,25.461538,18.384615,7.923077,3.076923,18.538462,17.692308,70.461538
4,DND,2018,15.0,3.0,31.000000,72.666667,6.466667,22.533333,10.466667,19.133333,12.066667,27.133333,12.333333,6.600000,3.000000,14.733333,16.466667,78.933333
5,DND,2019,15.0,5.0,27.466667,70.200000,5.866667,22.133333,9.533333,14.933333,10.666667,27.466667,11.200000,7.933333,2.266667,16.666667,16.133333,70.333333
6,HCW,2017,14.0,4.0,26.928571,65.071429,5.142857,17.142857,10.571429,17.857143,11.000000,28.642857,16.642857,8.571429,1.571429,23.714286,16.714286,69.571429
7,HCW,2018,14.0,1.0,29.857143,69.142857,6.928571,19.857143,10.214286,15.642857,9.000000,25.000000,11.142857,6.500000,1.857143,17.142857,15.785714,76.857143
8,HCW,2019,15.0,8.0,27.666667,73.466667,5.666667,24.800000,11.533333,17.933333,12.400000,28.466667,10.333333,12.066667,2.400000,17.266667,16.933333,72.533333
9,HNB,2017,14.0,7.0,24.642857,69.500000,6.500000,24.428571,13.428571,21.142857,9.785714,24.357143,15.142857,10.000000,2.428571,15.285714,17.000000,69.214286


### 2.1.2 Hollinger Advanced Stats
As per: http://www.espn.com/nba/hollinger/teamstats

In [7]:
adv_stats = ["wPct","poss","oPoss","pace","aRat","tRat","orr","drr","trr","ts","eFG","oEff","dEff","mov"]

df["wPct"] = df["win"]/df["gp"] #Win Percentage
df["poss"] = df["fga"] - df["oreb"] + df["tov"] + 0.4*df["fta"] #Estimate Possession

#Estimate Opponent Stats
oStats = pd.DataFrame(columns=["teamID","year","oPoss","oOreb","oDreb","oPts"]) #Opponent average
lStats = pd.DataFrame(columns=stats) #League Average
with pd.option_context('mode.chained_assignment', None):
    for i in [2017,2018,2019]:
        x = df[df["year"]==i]
        lStats = lStats.append(x[stats].mean(),ignore_index=True)
        n = x.shape[0]
        totPoss = x["poss"].sum()
        x["oPoss"]=(totPoss-x["poss"])/(n-1)
        totOreb = x["oreb"].sum()
        x["oOreb"]=(totOreb-x["oreb"])/(n-1)
        totDreb = x["dreb"].sum()
        x["oDreb"]=(totDreb-x["dreb"])/(n-1)
        totPts = x["pts"].sum()
        x["oPts"] = (totPts-x["pts"])/(n-1)
        oStats = oStats.append(x[["teamID","year","oPoss","oOreb","oDreb","oPts"]])
oStats = oStats.set_index(["teamID","year"])
df = df.join(oStats, on=["teamID","year"])

df["pace"] = 48 * ((df["poss"] + df["oPoss"]) / (2 * 48)) #Estimate Pace Factor
df["aRat"] = (df["ast"])*100/((df["fga"])+(df["fta"]*0.44)+(df["ast"])+(df["tov"])) #Assist Ratio
df["tRat"] = (df["tov"]*100)/((df["fga"])+(df["fta"]*0.44)+(df["ast"])+(df["tov"])) #TOV Ratio
df["orr"] = (df["oreb"]*48)/(48*(df["oreb"] + df["oDreb"])) #OffReb Rate
df["drr"] = (df["dreb"]*48)/(48*(df["dreb"] + df["oOreb"])) #DefReb Rate
df["trr"] = ((df["dreb"]+df["oreb"])*48)/(48*(df["dreb"]+df["oreb"] + df["oDreb"]+df["oOreb"])) #TotReb Rate
df["ts"] = (df["pts"])/(2*(df["fga"]+(df["fta"]*0.44))) #True Shooting %
df["eFG"] = (df["fgm"] + 0.5 * 3*df["x3pm"]) / df["fga"] #Effective FG%
df["oEff"] = df["pts"]/df["poss"]*100 #Offensive Efficiency / Rating
df["dEff"] = df["oPts"]/df["oPoss"]*100 #Defensive Efficiency / Rating
df["mov"] = df["pts"]-df["oPts"] #Margin of Victory 

df[["teamID","year"]+adv_stats]

,teamID,year,wPct,poss,oPoss,pace,aRat,tRat,orr,drr,trr,ts,eFG,oEff,dEff,mov
0,CTC,2017,0.571429,92.700000,85.856484,89.278242,17.135987,18.203474,0.305386,0.735752,0.519298,0.492666,0.592293,87.378641,85.243199,7.813187
1,CTC,2018,0.866667,84.200000,83.431619,83.815810,15.814659,13.363679,0.289453,0.719703,0.499714,0.529829,0.662455,101.821061,94.407510,6.967619
2,CTC,2019,0.642857,90.685714,83.669905,87.177810,13.210840,15.089446,0.266144,0.730172,0.508772,0.506271,0.591378,94.360428,87.017691,12.763810
3,DND,2017,0.153846,83.800000,87.636484,85.718242,16.289087,16.425397,0.262241,0.704555,0.478313,0.463920,0.510808,84.082981,85.916849,-4.832967
4,DND,2018,0.200000,82.986667,83.674286,83.330476,11.403703,13.622802,0.302820,0.724015,0.506959,0.486730,0.560092,95.115681,95.759066,-1.192381
5,DND,2019,0.333333,82.173333,85.372381,83.772857,10.703637,15.928031,0.256170,0.694170,0.469584,0.458074,0.516619,85.591433,88.852199,-5.521905
6,HCW,2017,0.285714,84.928571,87.410769,86.169670,14.691047,20.933165,0.287355,0.732472,0.512283,0.476983,0.532382,81.917578,86.342367,-5.901099
7,HCW,2018,0.071429,83.542857,83.563048,83.552952,10.682298,16.434304,0.241886,0.695291,0.464711,0.505468,0.582128,91.997264,96.383455,-3.683810
8,HCW,2019,0.533333,85.506667,84.705714,85.106190,9.483835,15.847182,0.287221,0.707757,0.490047,0.445770,0.492287,84.827694,89.032055,-2.881905
9,HNB,2017,0.500000,83.457143,87.705055,85.581099,13.863095,13.993879,0.258041,0.694697,0.467809,0.439161,0.494861,82.933927,86.134096,-6.329670


### 2.1.3 Search Widget

In [8]:
@interact
def search_by_val(teamID=list(df_team["teamID"])+["All"], year=[2017,2018,2019], statType=["Basic","Advanced"]):
    if teamID=="All":
        x = df[(df["year"]==year)].set_index("teamID")
        print("\n"+statType+" Statistics for all 6 teams in Regular Season "+str(year))
    else:
        x = df[(df["year"]==year) & (df["teamID"]==teamID)].set_index("teamID")
        print("\n"+statType+" Statistics for "+teamDict2[teamID]+" in Regular Season "+str(year))
    
    if statType=="Basic":
        return x[stats]
    if statType=="Advanced":
        return x[adv_stats]

interactive(children=(Dropdown(description='teamID', options=('DND', 'HNB', 'CTC', 'HCW', 'SGH', 'TLW', 'All')…

## 2.2 For Players

### 2.2.1 Basic Stats
Total Game Played, Total Win, Average Stats

In [9]:
x = pd.merge(result_player,df_round,on="roundID")
stats = ["fgm","fga","x3pm","x3pa","ftm","fta","oreb","dreb","ast","stl","blk","tov","pf","pts"]

gk = x[x["roundType"]=="Regular"].groupby(["teamID","roundYear","#"])
df2 = pd.DataFrame(columns=["teamID","year","#","name"]+stats)
for k, group in gk:
    y = group[stats].mean()
    y["teamID"]=k[0]
    y["year"]=k[1]
    y["#"]=k[2]
    y["name"]=group["name"].unique()[0]
    df2 = df2.append(y,ignore_index=True)

df2["name"] = df2["name"].str.replace("-"," ")
df2.head()

,teamID,year,#,name,fgm,fga,x3pm,x3pa,ftm,fta,oreb,dreb,ast,stl,blk,tov,pf,pts
0,CTC,2017,3,"Le, Nguycn",1.500000,3.500000,1.000000,2.250000,0.625000,0.875000,0.375000,1.000000,0.500000,0.625000,0.125000,1.250000,1.125000,4.625000
1,CTC,2017,4,"Thanh, Hucnh",0.750000,2.166667,0.083333,0.583333,0.000000,0.333333,0.333333,0.500000,1.833333,0.666667,0.000000,1.166667,0.583333,1.583333
2,CTC,2017,6,"Nguycn, Hoang",0.666667,1.222222,0.222222,0.666667,0.111111,0.333333,0.000000,0.555556,0.000000,0.000000,0.000000,0.555556,0.444444,1.666667
3,CTC,2017,7,"Du Minh, An",0.666667,2.000000,0.000000,0.444444,0.222222,0.444444,0.555556,0.555556,2.000000,1.000000,0.111111,1.555556,1.888889,1.555556
4,CTC,2017,8,"Ngcc, Nguycn",1.600000,4.200000,0.400000,1.900000,0.200000,0.500000,0.400000,1.700000,3.300000,0.600000,0.100000,3.400000,2.400000,3.800000


### 2.2.2 Hollinger Advanced Stats
As per: http://insider.espn.com/nba/hollinger/statistics

Since there was no data for minutes played, all players are qualified (Hollinger's rule: *To qualify: a player must have played 6.09 MPG.*)

In [10]:
adv_player_stats = ["aRat","tRat","ts","eFG","VOP","aPER"]

df2["aRat"] = (df2["ast"])*100/((df2["fga"])+(df2["fta"]*0.44)+(df2["ast"])+(df2["tov"])) #Assist Ratio
df2["tRat"] = (df2["tov"]*100)/((df2["fga"])+(df2["fta"]*0.44)+(df2["ast"])+(df2["tov"])) #TOV Ratio
df2["ts"] = (df2["pts"])/(2*(df2["fga"]+(df2["fta"]*0.44))) #True Shooting %
df2["eFG"] = (df2["fgm"]+0.5*3*df2["x3pm"])/df2["fga"] #Effective FG%
df2.head()

,teamID,year,#,name,fgm,fga,x3pm,x3pa,ftm,fta,...,ast,stl,blk,tov,pf,pts,aRat,tRat,ts,eFG
0,CTC,2017,3,"Le, Nguycn",1.500000,3.500000,1.000000,2.250000,0.625000,0.875000,...,0.500000,0.625000,0.125000,1.250000,1.125000,4.625000,8.873114,22.182786,0.595238,0.857143
1,CTC,2017,4,"Thanh, Hucnh",0.750000,2.166667,0.083333,0.583333,0.000000,0.333333,...,1.833333,0.666667,0.000000,1.166667,0.583333,1.583333,34.504391,21.957340,0.342219,0.403846
2,CTC,2017,6,"Nguycn, Hoang",0.666667,1.222222,0.222222,0.666667,0.111111,0.333333,...,0.000000,0.000000,0.000000,0.555556,0.444444,1.666667,0.000000,28.868360,0.608766,0.818182
3,CTC,2017,7,"Du Minh, An",0.666667,2.000000,0.000000,0.444444,0.222222,0.444444,...,2.000000,1.000000,0.111111,1.555556,1.888889,1.555556,34.775889,27.047913,0.354251,0.333333
4,CTC,2017,8,"Ngcc, Nguycn",1.600000,4.200000,0.400000,1.900000,0.200000,0.500000,...,3.300000,0.600000,0.100000,3.400000,2.400000,3.800000,29.676259,30.575540,0.429864,0.523810


### 2.2.3 Estimate PER
As per: https://www.basketball-reference.com/about/per.html

Since there was no data for minutes played, PERs are not scaled by *1/MP*

In [11]:
#Estimate PER Stats
lStats = pd.DataFrame(columns=["teamID","year","lgFT","lgPF","lgFTA","factor","VOP","DRB"]) #league average
with pd.option_context('mode.chained_assignment', None):
    for i in [2017,2018,2019]:
        x = df2[df2["year"]==i]
        x["lgFT"]=x["ftm"].mean()
        x["lgFTA"]=x["fta"].mean()
        x["lgFG"]=x["fgm"].mean()
        x["lgFGA"]=x["fga"].mean()
        x["lgTOV"]=x["tov"].mean()
        x["lgPF"]=x["pf"].mean()
        x["lgOREB"]=x["oreb"].mean()
        x["lgDREB"]=x["dreb"].mean()
        x["lgPTS"]=x["pts"].mean()
        x["lgAST"]=x["ast"].mean()
        x["factor"]=(2 / 3) - (0.5 * (x["lgAST"] / x["lgFG"])) / (2 * (x["lgFG"] / x["lgFT"]))
        x["VOP"]=x["lgPTS"] / (x["lgFGA"] - x["lgOREB"] + x["lgTOV"] + 0.44 * x["lgFTA"])
        x["DRB"]=x["lgDREB"]/(x["lgDREB"]+x["lgOREB"])
        lStats = lStats.append(x[["teamID","year","lgFT","lgPF","lgFTA","factor","VOP","DRB"]])
lStats = lStats.set_index(["teamID","year"])
subset = df[["teamID","year","ast","fgm","pace"]]
subset = subset.set_index(["teamID","year"])
lStats = lStats.join(subset, on=["teamID","year"]).rename(columns={"ast":"tAST","fgm":"tFGM","pace":"tPACE"})
df2 = df2.join(lStats,on=["teamID","year"],how="right").drop_duplicates()

df2["PER"] = (df2["x3pm"]
     + (2/3) * df2["ast"]
     + (2 - df2["factor"] * (df2["tAST"] / df2["tFGM"])) * df2["fgm"]
     + (df2["ftm"] *0.5 * (1 + (1 - (df2["tAST"] / df2["tFGM"])) + (2/3) * (df2["tAST"] / df2["tFGM"])))
     - df2["VOP"] * df2["tov"]
     - df2["VOP"] * df2["DRB"] * (df2["fga"] - df2["fgm"])
     - df2["VOP"] * 0.44 * (0.44 + (0.56 * df2["DRB"])) * (df2["fta"] - df2["ftm"])
     + df2["VOP"] * (1 - df2["DRB"]) * df2["dreb"]
     + df2["VOP"] * df2["DRB"] * df2["oreb"]
     + df2["VOP"] * df2["stl"]
     + df2["VOP"] * df2["DRB"] * df2["blk"]
     - df2["pf"] * ((df2["lgFT"] / df2["lgPF"]) - 0.44 * (df2["lgFTA"] / df2["lgPF"]) * df2["VOP"]))

adjusted = pd.DataFrame(columns=["teamID","year","lgPER","lgPACE"])
with pd.option_context('mode.chained_assignment', None):
    for i in [2017,2018,2019]:
        x = df2[df2["year"]==i]
        y = df[df["year"]==i]
        x["lgPER"]=x["PER"].mean()
        y["lgPACE"]=y["pace"].mean()
        x = x[["teamID","year","lgPER"]]
        y = y[["teamID","lgPACE"]]
        x["lgPACE"] = x["teamID"].map({list(y["teamID"])[i]:list(y["lgPACE"])[i] for i in range(6)})
        adjusted = adjusted.append(x)
adjusted = adjusted.set_index(["teamID","year"])
df2 = df2.join(adjusted,on=["teamID","year"],how="right").drop_duplicates()
df2["aPER"] = df2["PER"]*df2["lgPACE"]/df2["tPACE"]*15/df2["lgPER"]

df2.head()

,teamID,year,#,name,fgm,fga,x3pm,x3pa,ftm,fta,...,factor,VOP,DRB,tAST,tFGM,tPACE,PER,lgPER,lgPACE,aPER
0,CTC,2017,3,"Le, Nguycn",1.500000,3.500000,1.000000,2.250000,0.625000,0.875000,...,0.600327,0.832254,0.720865,21.785714,31.928571,89.278242,2.709779,3.873212,86.99707,10.226168
1,CTC,2017,4,"Thanh, Hucnh",0.750000,2.166667,0.083333,0.583333,0.000000,0.333333,...,0.600327,0.832254,0.720865,21.785714,31.928571,89.278242,1.290904,3.873212,86.99707,4.871613
2,CTC,2017,6,"Nguycn, Hoang",0.666667,1.222222,0.222222,0.666667,0.111111,0.333333,...,0.600327,0.832254,0.720865,21.785714,31.928571,89.278242,0.527944,3.873212,86.99707,1.992357
3,CTC,2017,7,"Du Minh, An",0.666667,2.000000,0.000000,0.444444,0.222222,0.444444,...,0.600327,0.832254,0.720865,21.785714,31.928571,89.278242,1.288189,3.873212,86.99707,4.861371
4,CTC,2017,8,"Ngcc, Nguycn",1.600000,4.200000,0.400000,1.900000,0.200000,0.500000,...,0.600327,0.832254,0.720865,21.785714,31.928571,89.278242,1.398077,3.873212,86.99707,5.276064


### 2.2.4 Search Widget

In [12]:
player_data = ["teamID","#","name"]

@interact
def search_player(teamID=list(df_team["teamID"])+["All"], year=[2017,2018,2019], statType=["Basic","Advanced"]):
    if teamID=="All":
        x = df2[(df2["year"]==year)]
        print("\n"+statType+" Statistics for all 6 teams players in Regular Season "+str(year))
    else:
        x = df2[(df2["year"]==year) & (df2["teamID"]==teamID)]
        print("\n"+statType+" Statistics for "+teamDict2[teamID]+" players in Regular Season "+str(year))
    
    if statType=="Basic":
        return x[player_data + stats]
    if statType=="Advanced":
        return x[player_data + adv_player_stats]

interactive(children=(Dropdown(description='teamID', options=('DND', 'HNB', 'CTC', 'HCW', 'SGH', 'TLW', 'All')…